In [11]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

cv10_df = pd.read_csv('ttest_10CV.csv')
cv10_ACC_df = cv10_df.iloc[:, 1::2]
cv10_ACC_df['Seed'] = cv10_df['Seed']
cv10_AUC_df = cv10_df.iloc[:, 2::2]
cv10_AUC_df['Seed'] = cv10_df['Seed']


# 1-tailed two sample welch t-test seed=123 code

Algorithm 1 and 2 10 fold CV results random variables $Y, X$. Let $Y_i, X_i$ be the 10 fold CV results with $i = 1,\ldots,10$. 

Let the population means of the two algorithms be $\mu_1$ and $\mu_2$ respectively. Then the test for statistical significance between distribution means is:

$\mathbf{H_0}: \mu_1 = \mu_2$
$\mathbf{H_a}: \mu_1 \neq \mu_2$ 

Test statistic T:

$$
T=\frac{\overline{Y}_{1}-\overline{Y}_{2}}{\sqrt{s_{1}^{2} / N_{1}+s_{2}^{2} / N_{2}}}
$$

$
\begin{array}{l}{\text { where } N_{1} \text { and } N_{2} \text { are the sample sizes, } \overline{Y}_{1} \text { and } \overline{Y}_{2} \text { are the sample means, and } s_{1}^{2}} {\text { and } s_{2}^{2} \text { are the sample variances. }}\end{array}
$

The test statistic is compared with the student t distribution with $N1 + N2 - 2$ degrees of freedom to obtain a p-value. 

In [12]:
norm_ttest = pd.DataFrame(columns=['model', 'MIDA', 'Raw', 'Parisot et al (1035)', 'Parisot et al (871)'])

# TP MIDA Ridge 1035 subjects
a = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP MIDA Ridge (ACC) (1035)']
b = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw Ridge (ACC) (1035)']
c = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018)   (ACC) (871)']

norm_ttest.loc[0] =['TP MIDA Ridge (1035)', 'NA',
                    ttest_ind(a, b, equal_var = False)[1]/2,
                    ttest_ind(a, c, equal_var = False)[1]/2,
                    ttest_ind(a, d, equal_var = False)[1]/2]

# TP raw Ridge 1035 subjects
a = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw Ridge (ACC) (1035)']
b = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw Ridge (ACC) (1035)']
c = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018)   (ACC) (871)']

norm_ttest.loc[1] =['TP raw Ridge (1035)', 'NA',
                    'NA',
                    ttest_ind(a, c, equal_var = False)[1]/2,
                    ttest_ind(a, d, equal_var = False)[1]/2]

# TP MIDA LR 871 subjects
a = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP MIDA LR (ACC) (871)']
b = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw LR (ACC) (871)']
c = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018)   (ACC) (871)']

norm_ttest.loc[2] =['TP MIDA LR (871)', 'NA',
                    ttest_ind(a, b, equal_var = False)[1]/2,
                    ttest_ind(a, c, equal_var = False)[1]/2,
                    ttest_ind(a, d, equal_var = False)[1]/2]

# TP raw LR 871 subjects
a = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw LR (ACC) (871)']
b = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw LR (ACC) (871)']
c = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018)   (ACC) (871)']

norm_ttest.loc[3] =['TP raw Ridge (871)', 'NA',
                    'NA',
                    ttest_ind(a, c, equal_var = False)[1]/2,
                    ttest_ind(a, d, equal_var = False)[1]/2]

# 1-tailed two sample "corrected resampled t-test" seed=123 code

Problem with the welch t-test to compare two algorithms (see [1]):

- individual 10 fold results are not independent since the training folds overlap
- the differences in model performance ($Y_i - Y_i$) are not normally distributed because the individual 10 fold results are not independent

Nadeau and Bengio (2003) (see [2]) propose the 'corrected resampled t test', which adjusts the variance based on overlap between the training/test sets. It has proper type 1 error, and greater statistical power (i.e. lower type 2 error) than the 5x2cv t test and McNemar's test

Test statistic:

$$
T=\frac{\frac{1}{n} \sum_{j=1}^{n} Y_j - X_j}{\sqrt{\left(\frac{1}{n}+\frac{n_{2}}{n_{1}}\right) \hat{\sigma}^{2}}}
$$

where $n_1$ and $n_2$ are the training and test fold sample sizes respectively. $n$ is the number of folds and $\hat{\sigma}^2$ is the sample variance of the differences $Y_j - X_j$.

The test statistic is compared to a student-t distribution with $n-1$ degrees of freedom.

References:

[1] http://rasbt.github.io/mlxtend/user_guide/evaluate/paired_ttest_kfold_cv/

[2] Nadeau, C. and Bengio, Y., 2000. Inference for the generalization error. In Advances in neural information processing systems (pp. 307-313).

In [13]:
from scipy.stats import t

# corrected resampled t-test function
def corr_ttest_fn(a, b, n, n1, n2, k=1, r=1):
    diff = a - b 
    diff_mean = diff.mean()
    diff_var = diff.var(ddof=1)
    denom = np.sqrt((1/(n*k*r) + n2/n1) * diff_var)
    t_stat = diff_mean/denom
    df = n*k*r - 1
    p = (1.0 - t.cdf(abs(t_stat), df))
    return p

In [14]:
# number of folds
n = 10
# number of training samples
n1 = 932
# number of test samples
n2 = 103

# TP MIDA Ridge 1035 subjects
corr_ttest = pd.DataFrame(columns=['model', 'MIDA', 'Raw', 'Parisot et al (1035)', 'Parisot et al (871)'])

a = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP MIDA Ridge (ACC) (1035)']
b = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw Ridge (ACC) (1035)']
c = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018)   (ACC) (871)']

corr_ttest.loc[0] =['TP MIDA Ridge (1035)', 'NA',
                    corr_ttest_fn(a, b, n, n1, n2),
                    corr_ttest_fn(a, c, n, n1, n2),
                    corr_ttest_fn(a, d, n, n1, n2)]

a = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw Ridge (ACC) (1035)']
b = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw Ridge (ACC) (1035)']
c = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018)   (ACC) (871)']

# TP raw Ridge 1035 subjects
corr_ttest.loc[1] =['TP raw Ridge (1035)', 'NA',
                    'NA',
                    corr_ttest_fn(a, c, n, n1, n2),
                    corr_ttest_fn(a, d, n, n1, n2)]

n = 10
n1 = 785
n2 = 86

# TP MIDA LR 871 subjects
a = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP MIDA LR (ACC) (871)']
b = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw LR (ACC) (871)']
c = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018)   (ACC) (871)']

corr_ttest.loc[2] =['TP MIDA LR (871)', 'NA',
                    corr_ttest_fn(a, b, n, n1, n2),
                    corr_ttest_fn(a, c, n, n1, n2),
                    corr_ttest_fn(a, d, n, n1, n2)]

# TP raw LR 871 subjects
a = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw LR (ACC) (871)']
b = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'TP raw LR (ACC) (871)']
c = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df.loc[cv10_ACC_df.Seed == 123, 'Parisot et al (2018)   (ACC) (871)']

corr_ttest.loc[3] =['TP raw Ridge (871)', 'NA',
                    'NA',
                    corr_ttest_fn(a, c, n, n1, n2),
                    corr_ttest_fn(a, d, n, n1, n2)]

# 1-tailed two sample “corrected repeated k-fold cv test” code


Bouckaert and Frank (2004) ([1]) extend the corrected resampled $k$-fold cv test to cases where $k$-fold cv is repeated $r$ times.

Test statistic:

$$
T=\frac{\frac{1}{k \cdot r} \sum_{i=1}^{k} \sum_{j=1}^{r} Y_{ij} - X_{ij}}{\sqrt{\left(\frac{1}{k \cdot r}+\frac{n_{2}}{n_{1}}\right) \hat{\sigma}^{2}}}
$$

Where $n_1$ and $n_2$ are the training and test fold sample sizes respectively. $k$ is the number of folds, $r$ is the number of repetitions and $\hat{\sigma}^2$ is the sample variance of the differences $Y_{ij} - X_{ij}$.

The test statistic is compared to a student-t distribution with $k\times r-1$ degrees of freedom.

References:

[1] Bouckaert, R.R. and Frank, E., 2004, May. Evaluating the replicability of significance tests for comparing learning algorithms. In Pacific-Asia Conference on Knowledge Discovery and Data Mining (pp. 3-12). Springer, Berlin, Heidelberg.

In [15]:
n = 1
n1 = 932
n2 = 103
# number of folds
k = 10 
# number of repetitions (different seeds)
r = 3

# TP MIDA Ridge 1035 subjects
r_corr_ttest = pd.DataFrame(columns=['model', 'MIDA', 'Raw', 'Parisot et al (1035)', 'Parisot et al (871)'])

a = cv10_ACC_df['TP MIDA Ridge (ACC) (1035)']
b = cv10_ACC_df['TP raw Ridge (ACC) (1035)']
c = cv10_ACC_df['Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df['Parisot et al (2018)   (ACC) (871)']

r_corr_ttest.loc[0] =['TP MIDA Ridge (1035)', 'NA',
                    corr_ttest_fn(a, b, n, n1, n2, k=k, r=r),
                    corr_ttest_fn(a, c, n, n1, n2, k=k, r=r),
                    corr_ttest_fn(a, d, n, n1, n2, k=k, r=r)]

# TP raw Ridge 1035 subjects
a = cv10_ACC_df['TP raw Ridge (ACC) (1035)']
b = cv10_ACC_df['TP raw Ridge (ACC) (1035)']
c = cv10_ACC_df['Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df['Parisot et al (2018)   (ACC) (871)']

r_corr_ttest.loc[1] =['TP raw Ridge (1035)', 'NA',
                    'NA',
                    corr_ttest_fn(a, c, n, n1, n2, k=k, r=r),
                    corr_ttest_fn(a, d, n, n1, n2, k=k, r=r)]

n = 1
n1 = 785
n2 = 86
k = 10
r = 3

# TP MIDA LR 871 subjects            
a = cv10_ACC_df['TP MIDA LR (ACC) (871)']
b = cv10_ACC_df['TP raw LR (ACC) (871)']
c = cv10_ACC_df['Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df['Parisot et al (2018)   (ACC) (871)']

r_corr_ttest.loc[2] =['TP MIDA LR (871)', 'NA',
                    corr_ttest_fn(a, b, n, n1, n2, k=k, r=r),
                    corr_ttest_fn(a, c, n, n1, n2, k=k, r=r),
                    corr_ttest_fn(a, d, n, n1, n2, k=k, r=r)]

# TP raw LR 871 subjects
a = cv10_ACC_df['TP raw LR (ACC) (871)']
b = cv10_ACC_df['TP raw LR (ACC) (871)']
c = cv10_ACC_df['Parisot et al (2018) (ACC) (1035)']
d = cv10_ACC_df['Parisot et al (2018)   (ACC) (871)']

r_corr_ttest.loc[3] =['TP raw Ridge (871)', 'NA',
                    'NA',
                    corr_ttest_fn(a, c, n, n1, n2, k=k, r=r),
                    corr_ttest_fn(a, d, n, n1, n2, k=k, r=r)]

## Notes

For the p-value results that follow, the following should be noted:

- Comparing performance between different sample sizes (1035, 871) is not valid since the data in individual folds is different. The p-values are given nonetheless.
- Comparison against *Raw* (see p value tables below) refers to the Raw version of the model in the model column

## Seed=123 average 10 fold results

In [16]:
result_summary = pd.DataFrame(cv10_ACC_df.loc[cv10_ACC_df.Seed == 123].mean(axis = 0))
result_summary.columns = ['ACC']
result_summary['AUC'] = list(cv10_AUC_df.loc[cv10_AUC_df.Seed == 123].mean(axis = 0))
result_summary = result_summary[:-1]
result_summary

,ACC,AUC
TP MIDA Ridge (ACC) (1035),0.727577,0.779279
TP raw Ridge (ACC) (1035),0.715935,0.782212
TP MIDA LR (ACC) (871),0.692327,0.752452
TP raw LR (ACC) (871),0.691124,0.750557
Parisot et al (2018) (ACC) (1035),0.683066,0.742288
Parisot et al (2018) (ACC) (871),0.703837,0.749609


### 1-tailed two sample welch t-test p values


In [17]:
norm_ttest

,model,MIDA,Raw,Parisot et al (1035),Parisot et al (871)
0,TP MIDA Ridge (1035),NA,0.241313,0.007615,0.110414
1,TP raw Ridge (1035),NA,NA,0.009999,0.226729
2,TP MIDA LR (871),NA,0.485272,0.340228,0.317500
3,TP raw Ridge (871),NA,NA,0.384607,0.330255


### 1-tailed two sample "corrected resampled t-test" p-values


In [18]:
corr_ttest

,model,MIDA,Raw,Parisot et al (1035),Parisot et al (871)
0,TP MIDA Ridge (1035),NA,0.217912,0.064961,0.157642
1,TP raw Ridge (1035),NA,NA,0.052201,0.305775
2,TP MIDA LR (871),NA,0.47047,0.387211,0.289592
3,TP raw Ridge (871),NA,NA,0.419651,0.339068


## Average 10 fold results over 3 seeds (12, 123, 1234)

In [19]:
result_summary = pd.DataFrame(cv10_ACC_df.mean(axis = 0))
result_summary.columns = ['ACC']
result_summary['AUC'] = list(cv10_AUC_df.mean(axis = 0))
result_summary = result_summary[:-1]
result_summary

,ACC,AUC
TP MIDA Ridge (ACC) (1035),0.726556,0.779243
TP raw Ridge (ACC) (1035),0.711747,0.778513
TP MIDA LR (ACC) (871),0.697315,0.758130
TP raw LR (ACC) (871),0.702608,0.764429
Parisot et al (2018) (ACC) (1035),0.684320,0.746053
Parisot et al (2018) (ACC) (871),0.685088,0.735512


### 1-tailed two sample “corrected repeated k-fold cv test” p-values

In [20]:
r_corr_ttest

,model,MIDA,Raw,Parisot et al (1035),Parisot et al (871)
0,TP MIDA Ridge (1035),NA,0.189157,0.029159,0.023766
1,TP raw Ridge (1035),NA,NA,0.037090,0.095330
2,TP MIDA LR (871),NA,0.351018,0.302337,0.243781
3,TP raw Ridge (871),NA,NA,0.248386,0.210891
